In [ ]:
import os
import re
import pandas as pd
from google.cloud import storage

# 例:
gcs_path = "gs://dena-ai-intern-yoshihara-data/pococha_comment_sorted"

def parse_gcs_path(gcs_path: str):
    """gs://bucket/prefix -> (bucket, prefix/) に分解"""
    if not gcs_path.startswith("gs://"):
        raise ValueError("gcs_path must start with 'gs://'")
    rest = gcs_path[5:]
    bucket, _, prefix = rest.partition("/")
    prefix = prefix.rstrip("/")
    if prefix:
        prefix = prefix + "/"
    return bucket, prefix

def list_gcs_files(gcs_path: str, recursive: bool = True):
    """
    指定パス配下の GCS オブジェクト一覧を取得。
    recursive=False なら“直下のみ”を列挙、True なら再帰的に列挙。
    """
    bucket_name, prefix = parse_gcs_path(gcs_path)
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    if recursive:
        # 全階層を列挙
        blobs_iter = client.list_blobs(bucket_name, prefix=prefix)
        files = [b.name for b in blobs_iter if not b.name.endswith("/")]
    else:
        # 直下のみを列挙（サブディレクトリは除外）
        blobs_iter = client.list_blobs(bucket_name, prefix=prefix, delimiter="/")
        files = [b.name for b in blobs_iter if not b.name.endswith("/")]

    # GCS 上のフル GCS パスに整形
    gcs_files = [f"gs://{bucket_name}/{name}" for name in files]
    return gcs_files

def extract_suffix_number(filename: str):
    """
    ベース名から拡張子直前の「_数字」を抽出。
      例: comment_68412208.csv -> 68412208
    見つからなければ None を返す。
    """
    base = os.path.basename(filename)
    m = re.search(r'_(\d+)(?=\.[^.]+$)', base)
    return int(m.group(1)) if m else None

# ========== ここから実行 ==========

# 1) ディレクトリ内のファイルを列挙してカウント
all_files = list_gcs_files(gcs_path, recursive=False)   # 直下のみなら recursive=False
print(f"ファイル数: {len(all_files)}")

# 2) 末尾「_数値」を抽出して CSV に保存
rows = []
for f in all_files:
    number = extract_suffix_number(f)
    rows.append({
        "gcs_path": f,
        "file_name": os.path.basename(f),
        "suffix_number": number
    })

df = pd.DataFrame(rows).sort_values(["suffix_number", "file_name"], na_position="last")
out_path = "pococha_comment_sorted_suffixes.csv"
df.to_csv(out_path, index=False, encoding="utf-8")
print(f"CSV を保存しました: {out_path}  （{len(df)} 行）")
